# Llama-Model

在transformers仓库中可以看到llama的源码，首先是LlamaModel类，继承自PreTrainedModel，这个类是所有模型的基类，包含了一些通用的方法，比如保存模型、加载模型、初始化权重等。

继承关系为：`LlamaModel` -> `LlamaPreTrainedModel` -> `PreTrainedModel`


## LlamaConfig

LlamaConfig 中主要是定义一些参数，比如vocab_size、hidden_size、num_hidden_layers、num_attention_heads等。所有的参数有默认值，可以直接创建cofing就能用。

```python
config = LlamaConfig()
```


# LlamaModel

## 初始化

- 设置了模型的两个属性:padding_idx（用于指定填充标记的索引），vocab_size（词汇表的大小）
- 初始化了模型的嵌入层、解码器层、归一化层
- 嵌入层（nn.Embedding）：模型使用嵌入层将输入的标记映射成密集的向量表示。
- 解码器层（nn.ModuleList()）：模型包含多个解码器层，这些层都是由 LlamDecoderLayer 定义
- 归一化层 LlamaRMSNorm：归一化层使用的是 Root Mean Square Layer Normalization（RMS Layer Norm）
- 设置了是否使用 gradient_checkpoint 主要是用来节省显存
- 调用 post_init() 完成一些初始化和准备检查的代码

```python
def __init__(self, config: LlamaConfig):
    super().__init__(config)
    self.padding_idx = config.pad_token_id
    self.vocab_size = config.vocab_size

    # embedding 层
    self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)
    # 中间的一堆 decoderlayers 层
    self.layers = nn.ModuleList(
        [LlamaDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
    )
    self._use_sdpa = config._attn_implementation == "sdpa"
    self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
    self.norm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    self.gradient_checkpointing = False
    # Initialize weights and apply final processing
    self.post_init()
```

可以看一下 `post_init()` 的代码，主要是初始化权重和`gradient_checkpointing`相关的一些事情。该方法在`PreTrainedModel`基类中，`transformers`中所有模型基本都继承这个类。
    
```python
def post_init(self):
    """
    A method executed at the end of each Transformer model initialization, to execute code that needs the model's
    modules properly initialized (such as weight initialization).
    """
    self.init_weights()
    self._backward_compatibility_gradient_checkpointing()
```

## forward

forward 部分的代码有点长，但其实大部分都是张量并行或者是节省显存相关的代码，对于理解模型结构来说可以直接忽略。

首先进来就是把 `inputs_ids` 进行向量化，然后拿到 `hidden_states` 。 然后是存起来所有的`hidden_states` 进入 `decoder_layer` 再拿一个 `hidden_states`，作为下一轮 `decoder_layer` 的 `hidden_states` 输入，最后给 `hidden_states` norm一下。 如下代码所示：

```python
inputs_embeds = self.embed_tokens(input_ids)
hidden_states = inputs_embeds

for decoder_layer in self.layers:
    # 存起来所有的 hidden_states
    if output_hidden_states:
        all_hidden_states += (hidden_states,)
    # 这里是 decoder_layer 的 forward
    layer_outputs = decoder_layer(
        hidden_states,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_value=past_key_values,
        output_attentions=output_attentions,
        use_cache=use_cache,
    )
    # 再拿一个 hidden_states，作为下一轮 decoder_layer 的 hidden_states 输入
    hidden_states = layer_outputs[0]

hidden_states = self.norm(hidden_states)
```

最后就是以 `BaseModelOutputWithPast` 的形式输出。ok，接下来继续看`decoder_layer`中的其他代码。



# LlamaDecoderLayer

Embedding层不用多说，用的就是torch中的nn.Embedding。那就直接来看DecoderLayer。

## DecoderLayers 初始化

先来看初始化。

- `hidden_size` : 也就是在上面说的输入输出。
- `self_attn` : 别看它写这么多啊，其实就是选一下用什么 `attention` 。看见大写字母不要怕，直接点进去看看怎么个事！
    ```python
    LLAMA_ATTENTION_CLASSES = {
        "eager": LlamaAttention,
        "flash_attention_2": LlamaFlashAttention2,
        "sdpa": LlamaSdpaAttention,
    }
    ```
- `mlp` : 一个全连接层 `LlamaMLP` 这个待会后面再说，输入输出都是 `hidden_size` 大小。
- `input_layernorm` : `LlamaRMSNorm` 层，输入时候的norm
- `post_attention_layernorm` : 丢入 `mlp` 之前的操作。

```python
class LlamaDecoderLayer(nn.Module):
    def __init__(self, config: LlamaConfig, layer_idx: int):
        super().__init__()
        self.hidden_size = config.hidden_size

        self.self_attn = LLAMA_ATTENTION_CLASSES[config._attn_implementation](config=config, layer_idx=layer_idx)

        self.mlp = LlamaMLP(config)
        self.input_layernorm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)\
```

## forward

首先复制一份 `hidden_states` 给 `residual`。然后 `hidden_states` 进入 `input_layernorm` 进行norm。然后进入 `self_attn` 进行 `attention` 操作，拿到 `hidden_states`、`self_attn_weights`、`present_key_value`。然后 `hidden_states` 和 `residual` 相加，得到 `hidden_states`。

然后 `hidden_states` 进入 `post_attention_layernorm` 进行norm。最后 `hidden_states` 进入 `mlp` 进行全连接操作，拿到 `hidden_states`。然后 `hidden_states` 和 `residual` 相加，得到 `hidden_states`。最后输出 `hidden_states`。

```python
residual = hidden_states

hidden_states = self.input_layernorm(hidden_states)

# Self Attention
hidden_states, self_attn_weights, present_key_value = self.self_attn(
    hidden_states=hidden_states,
    attention_mask=attention_mask,
    position_ids=position_ids,
    past_key_value=past_key_value,
    output_attentions=output_attentions,
    use_cache=use_cache,
    **kwargs,
)
hidden_states = residual + hidden_states

# Fully Connected
residual = hidden_states
hidden_states = self.post_attention_layernorm(hidden_states)
hidden_states = self.mlp(hidden_states)
hidden_states = residual + hidden_states

outputs = (hidden_states,)

if output_attentions:
    outputs += (self_attn_weights,)

if use_cache:
    outputs += (present_key_value,)

return outputs
```

# Attention